In [6]:
import OpenDartReader
import numpy as np
import cjw_maria
from pykrx import stock

dart = OpenDartReader('488d3a46f4b1d44b4be6197c7184603361289322')

host = "localhost"
user = "root"
password = "sa1234" 
db = "jamoo"

maria = cjw_maria.MariaDB(host, user, password, db)

class dartCollector:
    def __init__(self, nmid, datatype, tablename, fs):
        self.code_list = stock_amount_2016 = stock.get_market_cap_by_ticker("20161229 ")[['상장주식수']]
        self.code_list = list(stock_amount_2016.index)
        
        self.nmid = nmid
        self.datatype = datatype
        self.tablename = tablename
        self.fs = fs
        
    def play(self):
        if self.fs == 'fs' and self.datatype == 'id':
            print('fs일때는 datatype=nm 만 가능합니다')
            return
        self.done_data = maria.showData('select * from {}'.format(tablename))
        self.done_list = list(self.done_data['종목코드'])
        
        for code in self.code_list:
            self.code = code
            if code in self.done_list:
                continue
            else:
                try:
                    self.fs_list = self.getList(2020) + self.getList(2017)
                    self.saveList()
                except:
                    try:
                        self.fs_list =  self.getList(2019) + [-9999999]
                        self.saveList()
                    except:
                        continue
                        
    def getList(self, bsns_year):
        if bsns_year > 2017:
            want_list = ['bfefrmtrm_amount', 'frmtrm_amount', 'thstrm_amount']
        else:
            want_list = ['thstrm_amount']
        if self.fs == True:
            fs = dart.finstate(corp=self.code, bsns_year=bsns_year, reprt_code='11011')
        else:
            fs = dart.finstate_all(corp=self.code, bsns_year=bsns_year, reprt_code='11011')
        
        if self.datatype == 'nm':
            fs_list = list(fs[fs['account_nm']==self.nmid][want_list].iloc[0])
        elif self.datatype == 'id':
            fs_list = list(fs[fs['account_id']==self.nmid][want_list].iloc[0])
            
        fs_list = np.array(list(map(int, fs_list))) / 100000000
        fs_list = list(map(int, fs_list))
        return fs_list
    
    def saveList(self):
        self.fs_list = [self.code] + list(map(int, self.fs_list))
        maria.insertData(self.tablename, tuple(self.fs_list))
        maria.commitDB()
        


In [20]:
# nmid = '무형자산'
# datatype = 'nm'
# tablename = 'mh'
# dartcol = dartCollector(nmid, datatype, tablename)
# dartcol.play()

In [2]:
# nmid = 'dart_IssuedCapitalOfCommonStock' # 자본금
# datatype = 'id'
# tablename = 'jb'
# dartcol = dartCollector(nmid, datatype, tablename)
# dartcol.play()

In [97]:
# nmid = '무형자산'
# datatype = 'nm'
# tablename = 'mh'
# dartcol = dartCollector(nmid, datatype, tablename)
# dartcol.play()

In [7]:
nmid = '자본금'
datatype = 'nm'
tablename = 'jb'
fs = True

dartcol = dartCollector(nmid, datatype, tablename, fs)
dartcol.play()